# Предсказание стоимости жилья

## Введение

**Цель работы:** разработать две модели для предсказания медианной стоимости дома в жилом массиве. Первая модель со всеми данными, вторая - только с числовыми.

**Задачи:**
1. Обработать пропуски в данных.
2. Подготовить данные для моделей.
3. Обучить модели.
4. Оценить качество моделей по метрикам RMSE, MAE, R2.
5. Все работы проводить с помощью библиотеки Mlib в Spark.

**План работы:**
1. Загрузить данные.
2. Обработать пропуски.
3. Выполнить кодирование категориальных признаков.
4. Выполнить масштабирование числовых признаков.
5. Объединить обработанные данные.
6. Разделить данные на обучающую и тестовую выборки.
6. Обучить модели.
7. Получить получить предсказание моделей на тестовой выборке.
8. Рассчитать метрики RMSE, MAE, R2.
9. Сделать выводы по работе.

**Описание предоставленных данных**

В работе необходимо обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году.
В колонках датасета содержатся следующие данные:
- longitude — долгота;
- latitude — ширина;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану;
- rooms_per_household — отношение количества комнат total_rooms к количеству домовладений households (синтетический признак);
- population_in_household — отношение количества жителей population к количеству домовладений households (синтетический признак);
- bedroom_index — отношение количества спален total_bedrooms к общему количеству комнат total_rooms (синтетический признак).

## Подготовка данных

### Предварительная подготовка

Загрузим необходимые библиотеки и функции:

In [ ]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import (StringIndexer,
                                VectorAssembler,
                                StandardScaler,
                                Imputer)
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator,
                                   RegressionEvaluator)

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

RANDOM_SEED = 2022

In [ ]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California - Linear Regression") \
                    .getOrCreate()

Загрузим данные о жилье в Калифорнии

Загрузим таблицу:

In [ ]:
df_house = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

Добавим новые признаки:

In [ ]:
df_house = df_house.withColumn('rooms_per_household', F.col('total_rooms') / F.col('households'))
df_house = df_house.withColumn('population_in_household', F.col('population') / F.col('households'))
df_house = df_house.withColumn('bedroom_index', F.col('total_bedrooms') / F.col('total_rooms'))

Удалим ненужные признаки и сохраним датафрейм:

In [ ]:
exclude = ['longitude', 'latitude']
selected_columns = [col for col in df_house.columns if col not in exclude]
df_house = df_house.select(selected_columns)

Оценим типы данных в предоставленной таблице.

In [ ]:
df_house.printSchema()

root
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)



Посмотрим на первые 5 строчек в таблице:

In [ ]:
df_house.limit(5).toPandas()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household,bedroom_index
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,2.555556,0.146591
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,2.109842,0.155797
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,2.802260,0.129516
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,2.547945,0.184458
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,2.181467,0.172096


Посмотрим на количество пропусков в данных

In [ ]:
df_house.describe().toPandas()

,summary,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household,bedroom_index
0,count,20640,20640,20433,20640,20640,20640,20640,20640,20640,20640,20433
1,mean,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None,5.428999742190365,3.070655159436382,0.21303883048085015
2,stddev,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None,2.4741731394243205,10.38604956221361,0.05798267408098222
3,min,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN,0.8461538461538461,0.6923076923076923,0.1
4,max,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN,141.9090909090909,1243.3333333333333,1.0


Как видно из таблицы, пропуски есть только в столбце "total bedrooms" и соответственно в "bedroom_index". Их относительно немного, всего 207. Заполним пропуски после разделения на выборки.

Так как нам нужно обучить две модели, выделим отдельно датафрейм без категориального признака.

In [ ]:
df_house_num = df_house.select('housing_median_age',
                               'total_rooms',
                               'total_bedrooms',
                               'population',
                               'households',
                               'median_income',
                               'median_house_value',
                               'rooms_per_household',
                               'population_in_household',
                               'bedroom_index'
                              )

### Обработка данных

#### Данные для полного датафрейма.

Сперва подготовим данные для полного датафрейма.

Выделим отдельно числовые данные и колонку с целевым признаком.

In [ ]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['housing_median_age',
                   'total_rooms',
                   'total_bedrooms',
                   'population',
                   'households',
                   'median_income',
                   'rooms_per_household',
                  'population_in_household',
                  'bedroom_index']
target = 'median_house_value'

##### Обработка строковых данных

Разделим данные на тренировочную и тестовые выборки.

In [ ]:
train_data, test_data = df_house.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16418 4222


<span style="color: purple; font-weight: bold">Комментарий студента</span>
Спасибо!  

В данных у нас были пропуски. Заполним их.

In [ ]:
col_nan = ['total_bedrooms', 'bedroom_index']

Иницииализируем Imputer:

In [ ]:
imputer = Imputer(
    inputCols= col_nan,
    outputCols=col_nan,
    strategy="mean")

Обучим модель, заполним пропуски и оценим результат.

In [ ]:
model_nan = imputer.fit(train_data)

train_data = model_nan.transform(train_data)

train_data.describe().toPandas()

,summary,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household,bedroom_index
0,count,16418,16418,16418,16418,16418,16418,16418,16418,16418,16418,16418
1,mean,28.668169082714094,2632.2636131075647,537.3726081338816,1425.112681203557,498.68467535631623,3.862157820684621,206350.16134730174,None,5.431607696045633,3.0729293103710242,0.21317633730988578
2,stddev,12.637572573609434,2176.0356764859694,417.6006629159667,1137.2136474890342,379.7323946053574,1.9014322167050353,114986.77557779055,None,2.592331306041628,10.664926971799298,0.05791713129245083
3,min,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN,0.8461538461538461,0.6923076923076923,0.1
4,max,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN,141.9090909090909,1243.3333333333333,1.0


In [ ]:
test_data = model_nan.transform(test_data)
test_data.describe().toPandas()

,summary,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household,bedroom_index
0,count,4222,4222,4222,4222,4222,4222,4222,4222,4222,4222,4222
1,mean,28.527948839412602,2649.371387967788,539.7824844958841,1426.8924680246328,502.8645191852203,3.9037760303173825,208822.14874467076,None,5.418858248491711,3.061811717928805,0.21251085245965048
2,stddev,12.382058632796495,2203.384930352895,425.72751010375214,1113.9242505690397,392.29475369314537,1.8934067713363394,116964.86027913938,None,1.9479453366920383,9.222929437408906,0.05680763106513954
3,min,1.0,12.0,3.0,18.0,7.0,0.4999,14999.0,<1H OCEAN,0.8888888888888888,0.9705882352941176,0.1
4,max,52.0,30405.0,4457.0,12873.0,4204.0,15.0001,500001.0,NEAR OCEAN,41.333333333333336,599.7142857142857,0.896551724137931


Пропуски заполнены.

Преобразуем строковые данные в числовые в тренеровочной выборке и в тестовой выборке:

In [ ]:
indexer = StringIndexer(inputCols=categorical_cols,
                        outputCols=[c+'_idx' for c in categorical_cols])

In [ ]:
model_index = indexer.fit(train_data)

In [ ]:
train_data = model_index.transform(train_data)
cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|         INLAND|                1.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
+---------------+-------------------+
only showing top 3 rows



In [ ]:
test_data = model_index.transform(test_data)
cols = [c for c in test_data.columns for i in categorical_cols if (c.startswith(i))]
test_data.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|         INLAND|                1.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
+---------------+-------------------+
only showing top 3 rows



<span style="color: purple; font-weight: bold">Комментарий студента</span>
Спасибо!

Закодируем преобразованные данные:

In [ ]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])

In [ ]:
model_encoder = encoder.fit(train_data)

In [ ]:
train_data = model_encoder.transform(train_data)
cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



In [ ]:
test_data = model_encoder.transform(test_data)
cols = [c for c in test_data.columns for i in categorical_cols if (c.startswith(i))]
test_data.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|         INLAND|                1.0|      (4,[1],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Соберём признаки в один вектор:

In [ ]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data)

In [ ]:
test_data = categorical_assembler.transform(test_data)

Категориальные признаки обработаны.

##### Обработка числовых данных

Теперь масштабируем числовые данные:

In [ ]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

In [ ]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")

In [ ]:
model_scaler = standardScaler.fit(train_data)

In [ ]:
train_data = model_scaler.transform(train_data)

In [ ]:
test_data = model_scaler.transform(test_data)

Масштабирование проведено.

##### Сбор всех данных

Сперва ознакомимся с перечнем полученных столбцов:

In [ ]:
print(train_data.columns)

['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'rooms_per_household', 'population_in_household', 'bedroom_index', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']


In [ ]:
print(test_data.columns)

['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'rooms_per_household', 'population_in_household', 'bedroom_index', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']


Теперь соберем данные в одну таблицу:

In [ ]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features,
                                  outputCol="features")
train_data = final_assembler.transform(train_data)

train_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[1],[1.0])|     [0.07912912026224...|
|       (4,[1],[1.0])|     [0.07912912026224...|
|       (4,[1],[1.0])|     [0.07912912026224...|
+--------------------+-------------------------+
only showing top 3 rows



In [ ]:
test_data = final_assembler.transform(test_data)

test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[1],[1.0])|     [0.07912912026224...|
|       (4,[0],[1.0])|     [0.15825824052449...|
|       (4,[0],[1.0])|     [0.15825824052449...|
+--------------------+-------------------------+
only showing top 3 rows



Данные собраны и подготовлены.

#### Данные для датафрейма без категориального признака

Теперь подготовим данные для датафрейма без категориального признака.

Выделим отдельно числовые данные и колонку с целевым признаком.

In [ ]:
numerical_cols_num  = ['housing_median_age',
                   'total_rooms',
                   'total_bedrooms',
                   'population',
                   'households',
                   'median_income',
                   'rooms_per_household',
                  'population_in_household',
                  'bedroom_index']
target_num = 'median_house_value'

Разобьём данные на тренеровочную и тестовую выборки:

In [ ]:
train_data_num, test_data_num = df_house_num.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data_num.count(), test_data_num.count())

16418 4222


В данных были пропуски. Заполним их.

In [ ]:
model_nan = imputer.fit(train_data_num)

train_data_num = model_nan.transform(train_data_num)

train_data_num.describe().toPandas()

,summary,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,population_in_household,bedroom_index
0,count,16418,16418,16418,16418,16418,16418,16418,16418,16418,16418
1,mean,28.668169082714094,2632.2636131075647,537.3726081338816,1425.112681203557,498.68467535631623,3.862157820684621,206350.16134730174,5.431607696045633,3.0729293103710242,0.21317633730988578
2,stddev,12.637572573609434,2176.0356764859694,417.6006629159667,1137.2136474890342,379.7323946053574,1.9014322167050353,114986.77557779055,2.592331306041628,10.664926971799298,0.05791713129245083
3,min,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,0.8461538461538461,0.6923076923076923,0.1
4,max,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,141.9090909090909,1243.3333333333333,1.0


In [ ]:
test_data_num = model_nan.transform(test_data_num)
test_data_num.describe().toPandas()

,summary,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,population_in_household,bedroom_index
0,count,4222,4222,4222,4222,4222,4222,4222,4222,4222,4222
1,mean,28.527948839412602,2649.371387967788,539.7824844958841,1426.8924680246328,502.8645191852203,3.9037760303173825,208822.14874467076,5.418858248491711,3.061811717928805,0.21251085245965048
2,stddev,12.382058632796495,2203.384930352895,425.72751010375214,1113.9242505690397,392.29475369314537,1.8934067713363394,116964.86027913938,1.9479453366920383,9.222929437408906,0.05680763106513954
3,min,1.0,12.0,3.0,18.0,7.0,0.4999,14999.0,0.8888888888888888,0.9705882352941176,0.1
4,max,52.0,30405.0,4457.0,12873.0,4204.0,15.0001,500001.0,41.333333333333336,599.7142857142857,0.896551724137931


Пропуски обработаны.

##### Обработка числовых данных

Масштабируем числовые данные:

In [ ]:
numerical_assembler_num = VectorAssembler(inputCols=numerical_cols_num, outputCol="numerical_features")
train_data_num = numerical_assembler_num.transform(train_data_num)
test_data_num = numerical_assembler_num.transform(test_data_num)

In [ ]:
standardScalerNum = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")

In [ ]:
model_scaler_num = standardScalerNum.fit(train_data_num)

In [ ]:
train_data_num = model_scaler_num.transform(train_data_num)

In [ ]:
test_data_num = model_scaler_num.transform(test_data_num)

Масштабирование проведено.

##### Сбор всех данных

Сперва ознакомимся с перечнем полученных столбцов:

In [ ]:
print(train_data_num.columns)

['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'rooms_per_household', 'population_in_household', 'bedroom_index', 'numerical_features', 'numerical_features_scaled']


In [ ]:
print(test_data_num.columns)

['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'rooms_per_household', 'population_in_household', 'bedroom_index', 'numerical_features', 'numerical_features_scaled']


Теперь соберем данные в таблицу:

In [ ]:
all_features_num = ['numerical_features_scaled']

final_assembler_num = VectorAssembler(inputCols=all_features_num,
                                  outputCol="features")
train_data_num = final_assembler_num.transform(train_data_num)

train_data_num.select(all_features_num).show(3)

+-------------------------+
|numerical_features_scaled|
+-------------------------+
|     [0.07912912026224...|
|     [0.07912912026224...|
|     [0.07912912026224...|
+-------------------------+
only showing top 3 rows



In [ ]:
test_data_num = final_assembler_num.transform(test_data_num)

test_data_num.select(all_features_num).show(3)

+-------------------------+
|numerical_features_scaled|
+-------------------------+
|     [0.07912912026224...|
|     [0.15825824052449...|
|     [0.15825824052449...|
+-------------------------+
only showing top 3 rows



Данные собраны и подготовлены.

## Обучение моделей

### Модель на полных данных

Инициируем и обучем модель линейной регрессии:

In [ ]:
lr = LinearRegression(labelCol=target, featuresCol='features')

model = lr.fit(train_data)

24/08/29 03:44:57 WARN Instrumentation: [ba58489c] regParam is zero, which might cause numerical instability and overfitting.


Получим предсказание модели на тестовой выборке:

In [ ]:
predictions = model.transform(test_data)

predictedLabes = predictions.select('median_house_value', "prediction")
predictedLabes.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          191300.0| 171277.9575259618|
|          434700.0|138596.42735805726|
|          500001.0| 303974.9308022256|
|          315000.0|128695.08583627353|
|          100000.0|122034.22057770431|
|          136700.0| 168877.5521691165|
|          145600.0|198673.46251940067|
|          158500.0|233314.92089465223|
|          196700.0| 203176.1177894658|
|           96500.0|145857.13767816994|
|          111500.0|137865.33034596068|
|          137900.0| 135683.2909791577|
|          500001.0| 453030.0577048657|
|          204400.0| 241916.7338659679|
|          222500.0| 234452.9044352407|
|          187800.0|180648.87053012435|
|          220800.0|213189.77947124199|
|          151300.0| 188116.0217669895|
|          163500.0|117197.00666717239|
|          500001.0|  343253.709809841|
+------------------+------------------+
only showing top 20 rows



Предсказания получены.

### Модель на данных без категориального признака

Инициируем и обучем модель линейной регрессии:

In [ ]:
lr_num = LinearRegression(labelCol=target_num, featuresCol='features')

model_num = lr_num.fit(train_data_num)

24/08/29 03:44:58 WARN Instrumentation: [42448d8a] regParam is zero, which might cause numerical instability and overfitting.


Получим предсказание модели на тестовой выборке:

In [ ]:
predictions_num = model_num.transform(test_data_num)

predictedLabes_num = predictions_num.select('median_house_value', "prediction")
predictedLabes_num.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          191300.0|197626.37817900072|
|          434700.0| 86459.77837511533|
|          500001.0|274651.90501669364|
|          315000.0|133919.41642179058|
|          100000.0|141340.61950717217|
|          136700.0| 201797.9226816326|
|          145600.0|156676.05673911664|
|          158500.0|186729.71034591817|
|          196700.0|228194.95897920468|
|           96500.0|170584.82150010506|
|          111500.0|161844.62563444293|
|          137900.0| 159863.4813369382|
|          500001.0|  443096.304980543|
|          204400.0| 268803.9943450091|
|          222500.0|255669.73348247228|
|          187800.0|189103.97565617485|
|          220800.0| 234188.7192496979|
|          151300.0| 196987.2843804744|
|          163500.0|134214.84810127472|
|          500001.0|324097.61486691074|
+------------------+------------------+
only showing top 20 rows



Предсказания получены.

## Анализ результатов

Теперь оценим результаты работы модели по метрикам RMSE, MAE и R2. Сперва инициируем расчет метрик.

In [ ]:
rmse_evaluator = RegressionEvaluator(
    metricName="rmse", predictionCol="prediction", labelCol='median_house_value')
mae_evaluator = RegressionEvaluator(
    metricName="mae", predictionCol="prediction", labelCol='median_house_value')
r2_evaluator = RegressionEvaluator(
    metricName="r2", predictionCol="prediction", labelCol='median_house_value')

Теперь посчитаем метрики для моделей. RMSE:

In [ ]:
rmse = rmse_evaluator.evaluate(predictions)
rmse_num = rmse_evaluator.evaluate(predictions_num)
print(f'RMSE для модели на полных данных:{round(rmse,2)}.')
print(f'RMSE для модели без категориального признака:{round(rmse_num,2)}.')

RMSE для модели на полных данных:68885.09.
RMSE для модели без категориального признака:73954.32.


MAE:

In [ ]:
mae = mae_evaluator.evaluate(predictions)
mae_num = mae_evaluator.evaluate(predictions_num)
print(f'MAE для модели на полных данных:{round(mae,2)}.')
print(f'MAE для модели без категориального признака:{round(mae_num,2)}.')

MAE для модели на полных данных:50094.75.
MAE для модели без категориального признака:54597.28.


Метрика R2:

In [ ]:
r2 = r2_evaluator.evaluate(predictions)
r2_num = r2_evaluator.evaluate(predictions_num)
print(f'R2 для модели на полных данных:{round(r2,2)}.')
print(f'R2 для модели без категориального признака:{round(r2_num,2)}.')

R2 для модели на полных данных:0.65.
R2 для модели без категориального признака:0.6.


In [ ]:
spark.stop()

## Выводы

**В рамках работы сделано следующее:**

1. Обработаны пропуски в данных.
2. Подготовлены данные для моделей. Категориальные признаки закодированы, числовые признаки были масштабированы.
3. Обучены две модели: одна модель на полных данных, вторая модель на данных без категориальных признаков.
4. Работа моделей проанализирована по метрикам RMSE, MAE, R2.
Все работы проводили с помощью библиотеки Mlib в Spark. Необходимо отметить, что для обучения моделей использовались данные с синтетическими признаками: rooms_per_household, population_in_household, bedroom_index.

**Результаты работы:**  
Все три метрики у модели без категориального признака хуже, чем у модели, обученной на всех данных. Очевидно, учёт близости к океану играет важную роль в предсказании медианной стоимости дома в Калифорнии. Так, R2 у модели без категориального признака составляет 0,6, у модели на полных данных - 0,65.  

**Рекомендации:**  
Для улучшения точности предсказания рекомедуется добавить дополнительные признаки, к примеру:
- удаление от центра города;
- наличие объектов инфраструктуры;
- наличие метро, трамваем или другого общественного транспорта;
- другие особенности объектов недвижимости.
